# **Movie Recommendation System**

---

In [1]:
import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv("tmdb_5000_credits.csv")
movies_list = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")

In [3]:
ratings.head(1)

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044


In [4]:
ratings.shape

(25000095, 4)

In [5]:
movies_list.head(1)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [6]:
movies.shape

(4803, 20)

In [7]:
movies_list = movies_list.drop('genres', axis=1)
movies_list.head(1)

,movieId,title
0,1,Toy Story (1995)


In [8]:
movies_list['title'] = movies_list['title'].str.replace(r'\(\d{4}\)', '', regex=True).str.strip()
movies_list.head(2)

,movieId,title
0,1,Toy Story
1,2,Jumanji


In [9]:
ratings_with_movies = ratings.merge(movies_list, on='movieId')
ratings_with_movies.head(2)

,userId,movieId,rating,timestamp,title
0,1,296,5.0,1147880044,Pulp Fiction
1,3,296,5.0,1439474476,Pulp Fiction


In [10]:
ratings_with_movies.shape

(25000095, 5)

In [11]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [12]:
movies = movies[['genres', 'id', 'keywords', 'overview', 'title']]
movies.head(1)

,genres,id,keywords,overview,title
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Avatar


In [13]:
movies = movies.merge(movies_list, on='title')
movies.head(1)

,genres,id,keywords,overview,title,movieId
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...",Avatar,72998


In [14]:
movies.shape

(4323, 6)

In [15]:
ratings_with_movies = ratings_with_movies.drop_duplicates(subset=['userId', 'movieId', 'rating'])
ratings_with_movies.shape

(25000095, 5)

In [16]:
movies_ratings_all = movies.merge(ratings, on = 'movieId')
movies_ratings_with_newID = movies_ratings_all[['userId', 'id', 'title', 'rating']]
movies_ratings_with_newID.head(1)

,userId,id,title,rating
0,3,19995,Avatar,4.0


In [17]:
movies_ratings_with_newID.shape

(12680181, 4)

In [18]:
movies_ratings_with_newID = movies_ratings_with_newID.drop_duplicates(subset=['userId', 'id', 'rating'])
movies_ratings_with_newID.shape

(12664758, 4)

#### At this Point we have only two dataframe
    1. movies ratings with changed movie ID (rating_main)
    2. movies details (movies)

In [39]:
rating_main = movies_ratings_with_newID
rating_main.head(1)

,userId,id,title,rating
0,3,19995,Avatar,4.0


In [20]:
rating_main.shape

(12664758, 4)

In [21]:
movies = movies.drop('movieId', axis=1)
movies.head(1)
movies.shape

(4323, 5)

DATA PREPARATION DONE....

In [22]:
x = rating_main.groupby('userId').count()['rating'] > 200
genuine_user = x[x].index

In [23]:
filtered_rating = rating_main[rating_main['userId'].isin(genuine_user)]
filtered_rating.shape

(5363482, 4)

In [24]:
y = filtered_rating.groupby('title').count()['rating']>=50
famous_movies = y[y].index

In [25]:
final_ratings = filtered_rating[filtered_rating['title'].isin(famous_movies)]
final_ratings.shape

(5354017, 4)

In [26]:
user_movie_rating_table = final_ratings.pivot_table(index='id',columns='userId',values='rating')

In [27]:
user_movie_rating_table.fillna(0, inplace=True)

In [28]:
user_movie_rating_table

userId,3,12,13,31,43,57,72,75,80,120,...,162481,162484,162495,162508,162512,162516,162519,162521,162533,162534
id,,,,,,,,,,,,,,,,,,,,,
5,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0
12,4.0,4.0,4.0,3.0,4.0,4.5,0.0,0.0,0.0,0.0,...,4.0,3.0,1.0,5.0,4.5,4.0,0.0,4.0,4.5,4.0
13,4.0,4.0,5.0,3.0,5.0,4.0,5.0,2.0,4.0,5.0,...,4.5,3.0,5.0,5.0,0.0,4.5,0.0,3.5,4.5,2.5
14,5.0,4.0,4.0,3.0,5.0,5.0,5.0,3.5,4.0,5.0,...,4.0,3.5,4.0,0.0,4.0,5.0,5.0,0.0,4.0,4.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.5,0.0,0.0,0.0,4.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
374461,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
376659,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0


In [29]:
# final_ratings[final_ratings['title']=='Avatar']

In [30]:
from sklearn.metrics.pairwise import cosine_similarity

In [31]:
similarity_scores = cosine_similarity(user_movie_rating_table)
similarity_scores.shape

(2902, 2902)

In [32]:
# recommend('Pootie Tang')

In [33]:
movies.sample(5)

,genres,id,keywords,overview,title
1239,"[{""id"": 18, ""name"": ""Drama""}]",10802,"[{""id"": 246, ""name"": ""dancing""}, {""id"": 293, ""...",A young drifter named Nomi arrives in Las Vega...,Showgirls
1056,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 27, ""...",9360,"[{""id"": 5968, ""name"": ""amazon""}, {""id"": 10787,...","A ""National Geographic"" film crew is taken hos...",Anaconda
2202,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",50725,"[{""id"": 1412, ""name"": ""becoming an adult""}, {""...",Recent MIT grad Matt Franklin (Topher Grace) s...,Take Me Home Tonight
265,"[{""id"": 35, ""name"": ""Comedy""}]",38745,"[{""id"": 736, ""name"": ""journalist""}, {""id"": 369...",Travel writer Lemuel Gulliver takes an assignm...,Gulliver's Travels
3199,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 80, ""n...",334527,"[{""id"": 11578, ""name"": ""mobster""}, {""id"": 1882...",Four young men make a risky investment togethe...,Criminal Activities


In [34]:
!pip install python-Levenshtein

In [35]:
from fuzzywuzzy import process

In [36]:
def recommend_cf(movie_name):
    # Extract input movie ID
    movie_index = process.extractOne(movie_name, movies['title'])[2]
    movie_id = movies['id'].iloc[movie_index]
    
    # index fetch
    index = np.where(user_movie_rating_table.index==movie_id)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:5]
    recommended_movies = []
    for i in similar_items:
        item = []
        temp_df = movies[movies['id'] == user_movie_rating_table.index[i[0]]]
        recommedation = temp_df['title'].values[0]
        recommended_movies.append(recommedation)
        print(recommedation)
    
    return recommended_movies

In [38]:
# recommend_cf('Avatar')